In [ ]:
!unzip /content/mergeAttackFreeImperstion.zip

Archive:  /content/mergeAttackFreeImperstion.zip
  inflating: mergeAttackFreeImperstion.txt  


In [ ]:
!pip install -q torch-scatter -f https://pytorch-geometric.com/whl/torch-1.7.0+cu101.html
!pip install -q torch-sparse -f https://pytorch-geometric.com/whl/torch-1.7.0+cu101.html
!pip install -q torch-geometric

     |████████████████████████████████| 11.9MB 9.0MB/s 
     |████████████████████████████████| 24.3MB 118kB/s 
     |████████████████████████████████| 194kB 8.7MB/s 
     |████████████████████████████████| 235kB 14.4MB/s 
     |████████████████████████████████| 2.2MB 16.7MB/s 
     |████████████████████████████████| 51kB 9.5MB/s 


In [ ]:
import torch
from torch_geometric.data import Data

edge_index = torch.tensor([[0, 1, 3, 4],
                           [1, 0, 1, 1]], dtype=torch.long)
x = torch.tensor([[-1], [0], [1]], dtype=torch.float)

data = Data(x=x, edge_index=edge_index.t().contiguous())

print(data['x'])

tensor([[-1.],
        [ 0.],
        [ 1.]])


In [ ]:
import numpy as np

attackFreeArray = []
file = open('/content/mergeAttackFreeImperstion.txt', 'r')
timeList = []
counter = 1
graphList = []
ultimateGraphList = [] 
utlimateTimeList = []
all_unique_nodes = []
graphCounter = 0
noOfMessages = 200
for line in file:
  node = line.split()[0]

  if node == 'ID':
    continue

  graphList.append(node)
  timeList.append(counter)
  all_unique_nodes.append(node)

  if len(graphList) == noOfMessages:  #no of messages
    ultimateGraphList.append(graphList)
    utlimateTimeList.append(timeList)

    graphList = []
    timeList = []
   
  
  counter +=1



ugl = ultimateGraphList
udl = utlimateTimeList


print(len(ugl))

print(ugl[0])
print(len(ugl[0]))
print(len(utlimateTimeList[0]))
print(utlimateTimeList[0])
print(utlimateTimeList[3])




21865
['04b0', '05a0', '05a0', '02a0', '0260', '0080', '0081', '0165', '018f', '02b0', '0153', '0220', '0316', '0329', '0545', '043f', '0370', '0440', '0164', '04b1', '0316', '018f', '0260', '0080', '0081', '0165', '02a0', '02b0', '0329', '0153', '01f1', '0220', '0382', '0350', '04f2', '0545', '00a0', '00a1', '0370', '043f', '0440', '04f0', '05a2', '04f1', '05a2', '051a', '0690', '04b0', '0316', '018f', '0080', '0081', '0153', '0153', '0165', '0153', '0220', '0260', '02a0', '02b0', '0329', '0545', '043f', '0370', '0440', '0587', '05a0', '04b1', '0164', '0316', '0260', '02a0', '0080', '0081', '0164', '0165', '0153', '018f', '01f1', '0220', '02b0', '02c0', '0329', '0350', '0370', '0120', '0382', '0164', '043f', '0440', '04f2', '0517', '04f0', '0545', '05a2', '04b0', '0316', '0260', '018f', '0080', '0081', '0165', '0153', '01f1', '0220', '01f1', '02a0', '02b0', '0329', '0545', '043f', '0370', '0440', '0510', '04b1', '0164', '05e4', '0165', '02b0', '0080', '0081', '0153', '018f', '01f1', '

In [ ]:
#Making graph from the 200 sequential of id and store them in a file as Graph adjacent list
from collections import defaultdict
import networkx as nx
import matplotlib.pyplot as plt
import random
import torch
from scipy.sparse import csr_matrix
from torch_geometric.data import Data, DataLoader
gAttackfree=nx.DiGraph()
gAttacked = nx.DiGraph()

lenugl = len(ugl)

counter = 0
gAttackfreeCounter = 0
gAttackedCounter = 0



allLabelList = []
nodeFeatureList = []
uvList = []
DataList = []
for i in range(0,lenugl):
  uList = []
  vList = []
  attackFree = 0
  fList = []
  all_unique_nodes = list(set(ugl[i]))
  ln = len(list(set(ugl[i])))

  array = np.zeros((ln,ln))

  for j in range (0, len(ugl[i])-1):
     u = all_unique_nodes.index(ugl[i][j])
     v = all_unique_nodes.index(ugl[i][j+1])
     uList.append(u)
     vList.append(v)
     array[u][v] = array[u][v]+1
  uvList.append([uList,vList])
  edge_index =  torch.LongTensor([uList,vList])
  
  for k in range(len(array)):
    indeg = 0
    outdeg = 0
    for l in range(len(array[k])):
      outdeg = outdeg + array[k][l]
      indeg = indeg + array[l][k]
    fList.append([indeg,outdeg])
  
  nodeFeatureList.append(fList)
  x = torch.FloatTensor(fList)


  for j in range (0,len(ugl[i])):
    if str(ugl[i][j]) == '0164' and udl[i][j] >= 527250 and udl[i][j] <= 659990: 
      attackFree = 1
      break

  y = attackFree
  
  allLabelList.append(attackFree)

  DataList.append(Data(x = x,edge_index=edge_index,y=y))

  if y == 1:
    for m in range(30):
      DataList.append(Data(x = x,edge_index=edge_index,y=y))
      allLabelList.append(attackFree)





print(uvList[0])
print(allLabelList.count(0))
print(allLabelList.count(1))

print(nodeFeatureList[3])


loader = DataLoader(DataList, batch_size=32)

print(loader)
  






[[37, 18, 18, 6, 16, 24, 14, 32, 0, 9, 23, 25, 28, 33, 8, 21, 12, 34, 11, 17, 28, 0, 16, 24, 14, 32, 6, 9, 33, 23, 7, 25, 22, 36, 29, 8, 27, 1, 12, 21, 34, 35, 10, 3, 10, 13, 5, 37, 28, 0, 24, 14, 23, 23, 32, 23, 25, 16, 6, 9, 33, 8, 21, 12, 34, 2, 18, 17, 11, 28, 16, 6, 24, 14, 11, 32, 23, 0, 7, 25, 9, 31, 33, 36, 12, 26, 22, 11, 21, 34, 29, 15, 35, 8, 10, 37, 28, 16, 0, 24, 14, 32, 23, 7, 25, 7, 6, 9, 33, 8, 21, 12, 34, 30, 17, 11, 19, 32, 9, 24, 14, 23, 0, 7, 25, 25, 25, 16, 6, 28, 33, 36, 12, 22, 21, 34, 29, 8, 35, 37, 28, 16, 0, 24, 14, 32, 23, 25, 6, 9, 31, 31, 33, 8, 21, 12, 34, 20, 17, 11, 28, 16, 6, 24, 14, 32, 23, 0, 7, 25, 9, 33, 36, 22, 37, 37, 12, 21, 34, 29, 8, 35, 37, 28, 0, 24, 14, 32, 23, 25, 16, 6, 9, 33, 17, 17, 8, 21, 4], [18, 18, 6, 16, 24, 14, 32, 0, 9, 23, 25, 28, 33, 8, 21, 12, 34, 11, 17, 28, 0, 16, 24, 14, 32, 6, 9, 33, 23, 7, 25, 22, 36, 29, 8, 27, 1, 12, 21, 34, 35, 10, 3, 10, 13, 5, 37, 28, 0, 24, 14, 23, 23, 32, 23, 25, 16, 6, 9, 33, 8, 21, 12, 34, 2, 18, 

In [ ]:
import random 


torch.manual_seed(12345)
random.shuffle(DataList)

dataset = DataList


train_size = int(len(allLabelList)*.70)

#print(int(train_size))


train_dataset = dataset[:train_size]
test_dataset = dataset[train_size:len(allLabelList)]

print(f'Number of training graphs: {len(train_dataset)}')
print(f'Number of test graphs: {len(test_dataset)}')

Number of training graphs: 29249
Number of test graphs: 12536


In [ ]:
from torch_geometric.data import DataLoader

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset,batch_size=128, shuffle=False)

for step, data in enumerate(train_loader):
    print(f'Step {step + 1}:')
    print('=======')
    print(f'Number of graphs in the current batch: {data.num_graphs}')
    print(data)
    print()

Step 1:
Number of graphs in the current batch: 64
Batch(batch=[2045], edge_index=[2, 12736], x=[2045, 2], y=[64])

Step 2:
Number of graphs in the current batch: 64
Batch(batch=[2033], edge_index=[2, 12736], x=[2033, 2], y=[64])

Step 3:
Number of graphs in the current batch: 64
Batch(batch=[1936], edge_index=[2, 12736], x=[1936, 2], y=[64])

Step 4:
Number of graphs in the current batch: 64
Batch(batch=[2004], edge_index=[2, 12736], x=[2004, 2], y=[64])

Step 5:
Number of graphs in the current batch: 64
Batch(batch=[2033], edge_index=[2, 12736], x=[2033, 2], y=[64])

Step 6:
Number of graphs in the current batch: 64
Batch(batch=[2012], edge_index=[2, 12736], x=[2012, 2], y=[64])

Step 7:
Number of graphs in the current batch: 64
Batch(batch=[2033], edge_index=[2, 12736], x=[2033, 2], y=[64])

Step 8:
Number of graphs in the current batch: 64
Batch(batch=[2086], edge_index=[2, 12736], x=[2086, 2], y=[64])

Step 9:
Number of graphs in the current batch: 64
Batch(batch=[2033], edge_index

In [ ]:
from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.nn import global_mean_pool


class GCN(torch.nn.Module):
    def __init__(self, hidden_channels):
        super(GCN, self).__init__()
        torch.manual_seed(12345)
        self.conv1 = GCNConv(2, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.conv3 = GCNConv(hidden_channels, hidden_channels)
        self.lin = Linear(hidden_channels, 2)

    def forward(self, x, edge_index, batch):
        # 1. Obtain node embeddings 
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = self.conv2(x, edge_index)
        x = x.relu()
        x = self.conv3(x, edge_index)

        # 2. Readout layer
        x = global_mean_pool(x, batch)  # [batch_size, hidden_channels]

        # 3. Apply a final classifier
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin(x)
        
        return x

model = GCN(hidden_channels=64)
print(model)

GCN(
  (conv1): GCNConv(2, 64)
  (conv2): GCNConv(64, 64)
  (conv3): GCNConv(64, 64)
  (lin): Linear(in_features=64, out_features=2, bias=True)
)


In [ ]:
from IPython.display import Javascript
from sklearn.metrics import confusion_matrix
import numpy as np

display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))

model = GCN(hidden_channels=64)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.CrossEntropyLoss()
yTrue = []
yPred = []

def train():
    model.train()

    for data in train_loader:  # Iterate in batches over the training dataset.
         out = model(data.x, data.edge_index, data.batch)  # Perform a single forward pass.
         loss = criterion(out, data.y)  # Compute the loss.
         loss.backward()  # Derive gradients.
         optimizer.step()  # Update parameters based on gradients.
         optimizer.zero_grad()  # Clear gradients.

def test(loader):
     model.eval()

     correct = 0
     for data in loader:  # Iterate in batches over the training/test dataset.
         out = model(data.x, data.edge_index, data.batch)  
         pred = out.argmax(dim=1)  # Use the class with highest probability.
         correct += int((pred == data.y).sum())  # Check against ground-truth labels.
     return correct / len(loader.dataset)  # Derive ratio of correct predictions.


kkkk = 11
for epoch in range(1, kkkk):
    train()
    train_acc = test(train_loader)
    
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}')



<IPython.core.display.Javascript object>

Epoch: 001, Train Acc: 0.9336
Epoch: 002, Train Acc: 0.9366
Epoch: 003, Train Acc: 0.9345
Epoch: 004, Train Acc: 0.9370
Epoch: 005, Train Acc: 0.9360
Epoch: 006, Train Acc: 0.9373
Epoch: 007, Train Acc: 0.9322
Epoch: 008, Train Acc: 0.5421
Epoch: 009, Train Acc: 0.9378
Epoch: 010, Train Acc: 0.9357


In [ ]:
from sklearn.metrics import confusion_matrix

def conf(yTrue,yPred):
  tp,tn,fp,fn = 0,0,0,0
  for i in range(len(yTrue)):
    if yTrue[i] == yPred[i] and yTrue[i] == 1:
      tn= tn+1

    elif yTrue[i] == yPred[i] and yTrue[i] == 0:
      tp = tp+1
    elif yTrue[i] == 0 and yPred[i] == 1:
      fp=fp+1
    else:
      fn=fn+1
  

  return np.array([[tp,fp],[fn,tn]])
    




def test1(loader):
     model.eval()

     correct = 0
     cf = np.array([[0,0],[0,0]])
     for data in loader:  # Iterate in batches over the training/test dataset.
         out = model(data.x, data.edge_index, data.batch)  
         pred = out.argmax(dim=1)  # Use the class with highest probability.
         correct +=(data.y).sum()  # Check against ground-truth labels.
         c=conf(np.array(data.y),np.array(pred))
         print(c)

         cf = cf+c
     print(correct)
      

     return cf  # Derive ratio of correct predictions.

print('isafhksfh')
cf = test1(test_loader)  


cf = cf.T
print(cf)
print((cf[0][0]+cf[1][1])/(cf[0][0]+cf[0][1]+cf[1][0]+cf[1][1]))

isafhksfh
[[54  8]
 [ 0 66]]
[[58 11]
 [ 0 59]]
[[57  8]
 [ 0 63]]
[[66  7]
 [ 0 55]]
[[61  9]
 [ 0 58]]
[[60  6]
 [ 0 62]]
[[51  8]
 [ 1 68]]
[[57  7]
 [ 0 64]]
[[56  6]
 [ 0 66]]
[[61 14]
 [ 0 53]]
[[66  4]
 [ 1 57]]
[[58  8]
 [ 2 60]]
[[54  7]
 [ 1 66]]
[[58  6]
 [ 1 63]]
[[48  8]
 [ 0 72]]
[[55  6]
 [ 1 66]]
[[59  8]
 [ 0 61]]
[[62  5]
 [ 0 61]]
[[62  8]
 [ 0 58]]
[[50  4]
 [ 1 73]]
[[65  5]
 [ 0 58]]
[[51 12]
 [ 0 65]]
[[58 10]
 [ 0 60]]
[[68  6]
 [ 0 54]]
[[60  8]
 [ 2 58]]
[[54  8]
 [ 0 66]]
[[58  6]
 [ 1 63]]
[[47  9]
 [ 0 72]]
[[50 17]
 [ 0 61]]
[[50  4]
 [ 0 74]]
[[64  6]
 [ 1 57]]
[[45 16]
 [ 2 65]]
[[62  6]
 [ 0 60]]
[[57  9]
 [ 0 62]]
[[63  5]
 [ 0 60]]
[[55  5]
 [ 0 68]]
[[57  9]
 [ 0 62]]
[[64  7]
 [ 2 55]]
[[61  7]
 [ 1 59]]
[[57 10]
 [ 1 60]]
[[54 14]
 [ 0 60]]
[[56 11]
 [ 0 61]]
[[56  5]
 [ 0 67]]
[[57  4]
 [ 0 67]]
[[55  9]
 [ 1 63]]
[[53  7]
 [ 0 68]]
[[61  9]
 [ 0 58]]
[[52 13]
 [ 0 63]]
[[54  3]
 [ 1 70]]
[[46 17]
 [ 1 64]]
[[58  7]
 [ 1 62]]
[[60  6]
 [ 0 62]]
[[

In [ ]:
from sklearn.metrics import confusion_matrix
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np



cm = cf


cm_matrix = pd.DataFrame(data=cm, columns=["Attack free","Replay attacks"], 
                                 index=["Attack free","Replay attacks"])
sns.set(font_scale=2.0)
sns.heatmap(cm_matrix, annot=True, fmt= 'g', cmap='YlGnBu')
plt.rcParams['font.size'] = 3
plt.rcParams['figure.figsize'] = (3, 3)  
# plt.title("Confusion Matrix for All attack")
plt.xlabel("Actual label")
plt.ylabel("Predicted label")
plt.yticks(rotation = 45)
plt.xticks(rotation = 45, ha = 'right')
 
plt.savefig('confusion_replay.pdf', bbox_inches = "tight")
# plt.savefig('confusion_replay.svg', bbox_inches = "tight")
# plt.savefig('confusion_replay.png', bbox_inches = "tight")

In [ ]:
# cm = [[1612 ,  10],
#  [   5  ,793]]


tp = cm[0][0]
tn = cm[1][1]
fp = cm[0][1]
fn = cm[1][0]

re = tp/(tp+fn)
pr = tp/(tp+fp)
f1 = 2*pr*re/(pr+re)
fpr = fp / (fp+tn)
print("pr.........."+str(pr))
print("re----"+str(re))

print("f1===="+str(f1))
print("fpr====="+ str(fpr))
print((tp+tn)/(tp+tn+fp+fn))

pr..........0.98989898989899
re----0.9932432432432432
f1====0.9915682967959528
fpr=====0.009708737864077669
0.9917355371900827


In [ ]:
71002
170939
0.991287131986724
[[170760   1929]
 [   179  69073]]
 5

35791
85180
0.9906010531449686
[[84065    22]
 [ 1115 35769]]
 10


3659
8438
0.9962800694387038
[[8424   31]
 [  14 3628]]
100


1832
4217
0.993387336749876
[[4197   20]
 [  20 1812]]

803
1617
0.993801652892562
[[1612   10]
 [   5  793]]
 500



 464
746
0.9917355371900827
[[743   7]
 [  3 457]]
 1000



 309
296
0.9917355371900827
[[294   3]
 [  2 306]]
 2k

 175
67
1.0
[[ 67   0]
 [  0 175]]
 5k


 88
33
1.0
[[33  0]
 [ 0 88]]
 10k

In [ ]:
cm =[ [[170760 ,  1929],
 [179 , 69073]],

 [[84065   , 22],
 [ 1115, 35769]],

[[8424 ,  31],
 [  14, 3628]],

 [[4197 ,  20],
 [  20 ,1812]],

 [[1612  , 10],
 [   5  ,793]],

[[743 ,  7],
 [  3 ,457]],


 [[ 67 ,  0],
 [  0 ,175]],

 [[33,  0],
 [ 0, 88]]
]

for i in range(len(cm)):

  tp = cm[i][0][0]
  tn = cm[i][1][1]
  fp = cm[i][0][1]
  fn = cm[i][1][0]

  re = tp/(tp+fn)
  pr = tp/(tp+fp)
  f1 = 2*pr*re/(pr+re)
  fpr = fp / (fp+tn)


  mcc = (tp*tn-fp*fn)/((tp+fp)*(tp+fn)*(tn+fp)*(tn+fn))**.5

  fdr = fp/(fp+tp)

  # print("pr.........."+str(pr))
  # print("re----"+str(re))

  # print("f1===="+str(f1))
  # print("fpr====="+ str(fpr))
  print("mcccccc--------->" +str(mcc))
  #print("fdr======="+str(fdr))
  # print((tp+tn)/(tp+tn+fp+fn))

mcccccc--------->0.9789880325873227
mcccccc--------->0.9778659028555149
mcccccc--------->0.991178167512607
mcccccc--------->0.9843402613459972
mcccccc--------->0.9860106010676708
mcccccc--------->0.9825172905181248
mcccccc--------->1.0
mcccccc--------->1.0


In [ ]:
import pandas as pd

square_node_data = pd.DataFrame(
    {"x": [1, 2, 3, 4], "y": [-0.2, 0.3, 0.0, -0.5]}, index=["a", "b", "c", "d"]
)
square_node_data

,x,y
a,1,-0.2
b,2,0.3
c,3,0.0
d,4,-0.5


      Nodes  Edges  ...  MaximumPagerank  MininumPagerank
No.                 ...                                  
1        27    112  ...         0.215139         0.011104
2        25     75  ...         0.359649         0.018228
3        25     88  ...         0.285778         0.015716
4        27     97  ...         0.276848         0.014271
5        27    137  ...         0.173902         0.013195
...     ...    ...  ...              ...              ...
1828     26     70  ...         0.077329         0.013768
1829     26     73  ...         0.073358         0.011842
1830     26     78  ...         0.067820         0.011954
1831     26     85  ...         0.071132         0.009662
1832     26     76  ...         0.086512         0.008505

[1832 rows x 9 columns]
StellarGraph: Undirected multigraph
 Nodes: 1832, Edges: 0

 Node types:
  default: [1832]
    Features: float32 vector, length 9
    Edge types: none

 Edge types:
